In [24]:
from dwave.system import DWaveSampler, EmbeddingComposite
from neal import SimulatedAnnealingSampler
import numpy as np
import dimod

def graphcolour(G,ncolours, constr):
    #We start initialising the QUBO matrix
    Q = {}
    for i in range(0,len(G)):
        for j in range(0,ncolours):
            for k in range(0,len(G)):
                for l in range(0,ncolours):
                    Q[(str(i) + str(j), str(k) + str(l))] = 0
            
    #Objective, we dont want adjacent vertices to share the colour
    for i in range(0,len(G)):
        for j in range(0,ncolours):
            for k in range(0,len(G)):
                if G[i][j] == 1:
                    Q[(str(i) + str(j), str(k) + str(j))] = 1
    #Constraint: Every vertex is painted of exactly one colour
    for i in range(0,len(G)):
        for j in range(0,ncolours):
            for k in range(0,ncolours):
                if j == k:
                    Q[(str(i) + str(j), str(i) + str(j))] = Q[(str(i) + str(j), str(i) + str(j))] - constr
                else:
                    Q[(str(i) + str(j), str(i) + str(k))] = Q[(str(i) + str(j), str(i) + str(k))] + constr
    #sampler = SimulatedAnnealingSampler()
    sampler = EmbeddingComposite(DWaveSampler())
    sampleset = sampler.sample_qubo(Q, num_reads = 1000)
    return sampleset.first
                

In [25]:
G = ([0,1,1,1], [1,0,1,0],[1,1,0,1], [1,0,1,0])
ncolours = 3
print(graphcolour(G,3,10))

Sample(sample={'00': 1, '01': 0, '02': 0, '10': 0, '11': 1, '12': 0, '20': 1, '21': 0, '22': 0, '30': 0, '31': 1, '32': 0}, energy=-38.0, num_occurrences=1, chain_break_fraction=0.6666666666666666)


In [26]:
results = graphcolour(G,3,10)

In [27]:
results.sample

{'00': 1,
 '01': 0,
 '02': 0,
 '10': 0,
 '11': 1,
 '12': 0,
 '20': 0,
 '21': 0,
 '22': 1,
 '30': 0,
 '31': 1,
 '32': 0}